# Adiabatic flame temperature notebook.

Today, we are going to demonstrate the use of Cantera to calculate adiabatic flame temperatures for a range of fuels and conditions, and compare several methods in Cantera to how one would do it without Cantera.

## Adiabatic Flame Temperatures

As a bit of background, the adiabatic flame temperature is the temperature achieved when a given fuel-oxidizer combination is combustion in a perfectly insulated reactor. If no heat is lost to the surroundings and the reaction occurs at constant pressure, conservation of energy dictates that the reaction proceeds at constant enthalpy.  Namely, the enthalpy at the end of combustion (_state 2_) must equal that at the beginning (_state 1_):

$$ h_{\rm mix, 2} = h_{\rm mix, 1}$$

Wehn we consider the enthalpy of the mixture, there are two components to consider: the enthalpy of formation $\Delta^\circ_{\rm f}h$ and the sensible enthalpy $h_i$.

To make this more concrete, let's pick a specific reaction: stoichiometric combustion of methane, CH$_4$ with air::
$${\rm CH_4 + 2\left(O_2 + 3.76\,N_2\right) \leftrightharpoons 2\,H_2O + CO_2 + 7.52\,N_2 }$$

From the reaction stoichiometry (note that the net stoichiometric coefficient for N$_2$ is zero), the heat of combustion released by this reaction is:
$$ q_{\rm rxn} = -\sum_k \nu_k \Delta^\circ_{{\rm f}}h_k = \Delta^\circ_{\rm f}h_{\rm CH_4} + 2\Delta^\circ_{\rm f}h_{\rm O_2} - \Delta^\circ_{\rm f}h_{\rm CO_2}-2 \Delta^\circ_{\rm f}h_{\rm H_2O}$$

Since the reaction must occur at constant enthalpy, the heat released must be stored in the products. In essence, the heat is used to heat the products to a final temperature:
$$q_{\rm heating} = \sum_k\nu_{{\rm prod},k}\left[h_k(T_2) - h_k(T_1)\right] = h_{\rm CO_2}(T_2) - h_{\rm CO_2}(T_1) + 2\left[h_{\rm H_2O}(T_2) - h_{\rm H_2O}(T_1)\right] + 7.52\left[h_{\rm N_2}(T_2) - h_{\rm N_2}(T_1)\right] $$
where, again, states 1 and 2 represent the final and initial states, respectively.  $\nu_{\rm prod}$ is the reverse stoichiometric coefficient (i.e. that for all product species). Note that this assumes complete combution - there are no reactant species left to heat.

For a purely adiabatic reaction, we will have $q_{\rm rxn} = q_{\rm heating}$, and hence:
$$\sum_k\nu_{{\rm prod},k}\left[h_k(T_2) - h_k(T_1)\right] + \sum_k \nu_k \Delta^\circ_{{\rm f}}h_k = 0$$
or:
$$h_{\rm CO_2}(T_2) - h_{\rm CO_2}(T_1) + 2\left[h_{\rm H_2O}(T_2) - h_{\rm H_2O}(T_1)\right] + 7.52\left[h_{\rm N_2}(T_2) - h_{\rm N_2}(T_1)\right] + \Delta^\circ_{\rm f}h_{\rm CO_2}-2 \Delta^\circ_{\rm f}h_{\rm H_2O} - \Delta^\circ_{\rm f}h_{\rm CH_4} + 2\Delta^\circ_{\rm f}h_{\rm O_2} = 0$$

Note that only a few variables are dependent on the final temperature $T_2$.  We might be tempted to write the above equation in terms of those variables, but let's hold off for now. Our next step really depends on what method we use to solve the problem. In short, we need a means of calculating the enthalpy values, so that we may determine which value of $T_2$ satisfies our adiabatic condition.  

## Approach 1 - Complete combustion, calculating "by hand" and using tabulated data.

One popular source for these values are the [JANAF tables](https://janaf.nist.gov/), a thermodynamic database maintained by NIST.  These provide a range of thermodynamic properties, including enthalpis of formation, total sensible enthalpy, specific, and others, for a range of temperatures and for a wide variety of species.  For the curious, a collection of these tables are stored in the `data` folder that accompanies this notebook.

We will take the enthalpies of formation at the initial condition of 298.15 K:

| Species $k\hspace{3em}$ | $\Delta^\circ_{\rm f}h_k(298.15 K)\,\frac{\rm kJ}{\rm mol}$|
|---|---|
|CH$_4$| -74.873 |
| O$_2$| 0.0 |
| N$_2$| 0.0 |
| CO$_2$| -393.522 |
| H$_2$O | -241.826|

From these, let's solve for the heat released by the reaction.  Give it the variable name `q_rxn`:

In [12]:
# Entahlpies of formation, kJ/mol
h_f_CH4 = -74.873
h_f_CO2 = -393.522
h_f_H2O = -241.826
h_f_O2 = 0

q_rxn = h_f_CH4 + 2*h_f_O2 - h_f_CO2 - 2*h_f_H2O


In [13]:
print('The heat of reaction is {:.2f} kJ/mol.'.format(q_rxn))

The heat of reaction is 802.30 kJ/mol.


This heat must be stored in the sensible energy of the products, which reach an elevated temperature $T_2$, after combustion.

To calculate the final temeprature, we need to estimate how the species enthalpies change as a function of temperature. The JANAF tables actually give us $h_k(T) - h_k(298.15\,{\rm K})$ as a function of $T$, allowing for a high-precision solution.  This is demosntrated in the `adiabatic-flame-detailed.ipynb` notebook in this repo.

However, a simpler and nearly as acurrate solution uses the definition of the specific heat to calculate the change in enthalpy: 
$$ h_k(T_2) - h_k(T_1) = \int_{T_1}^{T_2}C_p dT$$
If $C_p$ is constant, this reduces to:
$$ h_k(T_2) - h_k(T_1) = C_p\left(T_2 - T_1\right)$$
While $C_p$ is not in fact constant, we can take the value at some intermediate temperature and recognize that the variation in $C_p$ from this average value is not _too_ significant.

Finally, to simplify a little bit further, let us recognize that, on a molar basis, our products are $\frac{7.52}{10.52} = 71.5%$ nitrogen.  So let us consult the JANAF tables once more, and choose the $C_p$ of N$_2$ at $T_{\rm avg} = 1000$ K (a guess value) as our $C_{p,{\rm avg}}$ for the mixture.
$$C_{p,{\rm avg}} = 32.697\,\frac{\rm J}{\rm mol-K}$$

Thefefore, $q_{\rm heating} = C_{p,{\rm avg}}\,n_{\rm tot}\left(T_2 - T_1\right)$, where $n_{\rm tot} = 10.52$, and we can therefore solve directly for $T_2$:

$$T_2 = T_1 + \frac{q_{\rm rxn}}{n_{\rm tot}C_{p,{\rm avg}}}$$

In [14]:
T_1 = 298.15 # Initial temperature, K
C_p_avg = 32.697*10**(-3) #Convert to kJ/mol-K
n_tot = 10.52

T_2 = T_1 + q_rxn/n_tot/C_p_avg
print('Initial T_ad = {:.2f} K.'.format(T_2))
print('T_avg = {:.2f}'.format(0.5*(T_1+T_2)))

Initial T_ad = 2630.61 K.
T_avg = 1464.38


Not bad!  If this is correct, our average temperature would $T_{\rm avg} = \frac{298.15 + 2630.61\,{\rm K}}{2}= 1464.4$ K.  So our average $C_p$ was taken at a temperature that is perhaps a bit low.  What if we re-did it, using $C_{p,\,{\rm N_2}}(1400\,{\rm K})=34.518\,\frac{\rm J}{\rm mol-K}$? 

Right away, we can see that $C_p$ hasn't changed all _that_ much, in going from 1000 K to 1400 K.  But let's check, anyway:

In [15]:
C_p_avg = 34.518*10**(-3) #Convert to kJ/mol-K

T_2 = T_1 + q_rxn/n_tot/C_p_avg
print('Initial T_ad = {:.2f} K.'.format(T_2))
print('T_avg = {:.2f}'.format(0.5*(T_1+T_2)))


Initial T_ad = 2507.56 K.
T_avg = 1402.85


We see that the adiabatic flame temperature estimate _did_ drop by 123 K, and now our actual average temperature is quite close to our guess value.

Again a still more accurate approach would be to use the species enthalpy values directly, and search the tabulated data to find a $T_2$ that satisfies our adiabatic condition.  This involves a _slightly_ more sophisticated coding approach, and is left as an extension code (`adiabatic-flame-detailed.ipynb`) for you to explore on your own.

We will note here that the detailed approach returns an adiabatic flame temperature of $T_{\rm ad} = 2327$ K, which is not _too_ far off from our rather simple estimate, here!

## Approach 2: complete combustion, using Cantera.
We will now show how to use Cantera to do this same calculation, using simple function calls to automate and generalize the process.  We will first repeat the "complete combustion" calcuation above, and then demonstrate a more realistic case with an array of intermediate and minor species.  

To skip to the end for a minute, we are going to find the adiabatic flame temperature by asking cantera to find the equilibrium state of a fuel-air mixture held at fixed enthalpy and pressure.  The function call will look like this:

> ``` gas.equilibrate('HP')```

Let's look at the individual pieces of this function call.  Cantera's source code is in C++, and calculations are based on user-created ''phase'' objects.  A phase object is an instantiation of one of Cantera's C++ classes, which includes any number of methods (i.e. functions) belonging to that class.

In the example above, we have created an ideal gas object called gas.  Among the methods defined by the Cantera source code for this class (and others) is an `equlibrate` function, which we call. The `equilibrate` routine requires two intensive mixture-average properties be held constant, which we specify via the string 'HP'.

But how do we create the gas object?  And how do we set the correct conditions so that the _correct_ enthalpy and pressure values are used?  In all cases, performing calculations with Cantera involves three basic steps:

- Identify or develop a YAML-based input file.
- Create Cantera phases and set initial conditions.
- Run the calculation.

### Cantera input files

### Create Cantera phases and set initial conditions

Okay, now that we have created our input file `methane.yaml`, we can create the phase object in Python, using Cantera routines.


In [16]:
import cantera as ct

gas = ct.Solution('input-files/methane.yaml')

Again, this has created a Cantera class object with the Python variable name `gas`, representing an ideal gas phase object.

We next want to set the initial conditions:

- Temperature = 298 K
- Stoichiometric mixture of methane and air: 1 mole of CH$_4$ to 2 moles of (O$_2$ + 3.76 N$_2$)
- One atmosphere of pressure.

To do this, we will use the Cantera method `TPX`, which is a member of our ideal gas class (and nearly all other Cantera phase classes). The syntax looks like this:
> ``` gas.TPX = T_value, P_value, molefractions```
where `T_value`, `P_value`, and `molefractions` are variables we create to specify the relevant values.  

Temperature and pressure are relatively straightforwad: we set the numerical values in units of K and Pa, respectively.  For pressure, there is even a cantera function `one_atm` to give use the value of one standard atmosphere in Pa (i.e. `ct.one_atm = 101325`).

For the mole fraction values, we have two options.  We can pass an array of mole fractions, in the exact order expected by Cantera, or we can pass a string that specifies the mole fractions of individual species, in any order.  For either option, Cantera's default behavior is to normalize the values so that the sum of all mole fractions equals one, so that only the relative values matter (note: this default normalization can be disabled, when needed).

We will choose the latter option, where the species names and mole fractions are separated by a colon, and entries for each species are separated by commas:
>```gas.TPX = 298, ct.one_atm, 'CH4:1.0, O2:2.0, N2:7.52'```

Note that, even here, there are other ways to set this strin (e.g. using Python string formatting) which can be helpful when trying to generalize your code or base your string input on the results of some other calculation.


In [17]:
gas.TPX = 298, ct.one_atm, 'CH4:1.0, O2:2.0, N2:7.52'

We can verify that we set the state as intended by printing out the state of the `gas` object:

In [18]:
gas()


  methane-air:

       temperature   298 K
          pressure   1.0132e+05 Pa
           density   1.1301 kg/m^3
  mean mol. weight   27.633 kg/kmol
   phase of matter   gas

                          1 kg             1 kmol     
                     ---------------   ---------------
          enthalpy       -2.5674e+05       -7.0947e+06  J
   internal energy        -3.464e+05       -9.5724e+06  J
           entropy            7240.5        2.0008e+05  J/K
    Gibbs function       -2.4144e+06       -6.6719e+07  J
 heat capacity c_p            1076.9             29758  J/K
 heat capacity c_v            775.99             21443  J/K

                      mass frac. Y      mole frac. X     chem. pot. / RT
                     ---------------   ---------------   ---------------
               CH4          0.055187          0.095057           -54.877
                O2           0.22014           0.19011           -26.334
                N2           0.72467           0.71483           -2

### Run the calcualtion:

Okay, so now all that is left is to run our calculation, and ask Cantera to give us the temperature of the equilibrated state.

Again, this is done via the `equilibrate` function:

>```gas.equlibrate('HP')```

We can then report out the adiabatic flame temperature by calling the gas object's temperature method:

>```gas.T```

In [19]:
gas.equilibrate('HP')
T_ad = gas.T

In [20]:
print('The adiabatic flame temperature is T_ad = {:.2f} K.'.format(T_ad))
print('The adiabatic flame temperature from our hand calculation was T_ad = {:.2f} K.'.format(T_2))

The adiabatic flame temperature is T_ad = 2325.49 K.
The adiabatic flame temperature from our hand calculation was T_ad = 2507.56 K.


So our result is quite close to that reported by the "detailed" method, but without any of the coding required in that notebook.  Let's look at the final, equilibrated state:

In [21]:
gas()


  methane-air:

       temperature   2325.5 K
          pressure   1.0133e+05 Pa
           density   0.14481 kg/m^3
  mean mol. weight   27.633 kg/kmol
   phase of matter   gas

                          1 kg             1 kmol     
                     ---------------   ---------------
          enthalpy       -2.5674e+05       -7.0947e+06  J
   internal energy       -9.5644e+05        -2.643e+07  J
           entropy              9867        2.7266e+05  J/K
    Gibbs function       -2.3202e+07       -6.4116e+08  J
 heat capacity c_p              1524             42113  J/K
 heat capacity c_v            1223.1             33799  J/K

                      mass frac. Y      mole frac. X     chem. pot. / RT
                     ---------------   ---------------   ---------------
               CH4        6.3133e-08        1.0875e-07           -50.584
                O2        2.5184e-07        2.1749e-07           -44.654
               H2O           0.12394           0.19011         

Note that not _all_ of the methane and oxygen are consumed, in this equilibrium calculation (this would consitute a very low entropy state, and therefore is not favored by thermodynamics).  Note also that Cantera has preserved the relative amounts of each _element_.  We can quickly verify that 3 moles of reactants (CH$_4$ + 2 O$_2$) makes three moles of product (2 H$_2$O + CO$_2$), which means that our N$_2$ mole fraction is unchanged, before and after the reaction.

## Approach 3 - Complex combustion, using Cantera

Note also that our flame has only two product species (plus nitrogen), because that is all we have _allowed_; our input file lists only five species, so those are the only five species considerd here (no piece of code will ever do anything we have not explicitly told it to do, wittingly or not!).

In real combustion applications, of course, a wide array of minor product species and side reactions are possible that can impact $T_{ad}$ in ways that we might care about.  How can we take a look at those?

Using the three steps from above:

- Identify or develop a YAML-based input file.
- Create Cantera phases and set initial conditions.
- Run the calculation.

The only step requiring modification is the first: we need a new input file to represent a more complex combustion chemistry.

### Cantera input files, revisited.

#### Mechanism files that come with Cantera.

From our previous input file, `methane.yaml`, we know that Cantera comes with a small collection of pre-defined iput files, and that one of these is `gri30.yaml`, which represents the GRI-3.0 mechanism for natural gas combustion developed in the 1970s and last updated in the 1990s.  In most cases, these mechanism files are included in Cantear for demonstration purposes only, and are not meant as an endoresement of that particular chemistry (this is partiulcarly true in the case of `gri30.yaml`).

We will plot adiabatic flame temperatures based on `gri30.yaml`, but: new and improved mechanisms are constantly developed and reported in the literature, representing the best-available data and community understanding.  How can we use these calculations in Cantera?

#### Conversion from Chemkin format
Most commonly, mechanisms are published and made available in a format consistent with the Chemkin software suite.  These input files can easily be converted to Cantera format, using the script `ck2yaml` (`ck` = "Chemkin").

We have downloaded the mechanism files for one such mechanism, the `NUIGMech1.1` from the National University of Ireland, Galway (http://www.nuigalway.ie/combustionchemistrycentre/mechanismdownloads/). These files are stored in the folder `input-files/NUIG`.

We will use the `ck2yaml` Python script file from the command line to convert these files into YAML format. Navigate to the directory `input-files/NUIG` and run the code:

```ck2yaml --input=NUIGMECH_1.1_HT.MECH, --thermo=NUIGMech1.1.THERM, --transport=NUIGMech1.1.TRAN --output=nuig_11.yaml```

This will result in several error messages and a failure to convert.  These mechanism files have hundreds of species and sometimes 1000s reactions; errors can be quite common! We have catalogued some of the most common conversion errors, so that you can debug as necessary, available [here](https://cantera.org/tutorials/ck2yaml-tutorial.html#debugging-common-errors-in-ck-files).

One thing we can try is re-running the conversion script with the `permissive` option:

```ck2yaml --input=NUIGMECH_1.1_HT.MECH, --thermo=NUIGMech1.1.THERM, --transport=NUIGMech1.1.TRAN --output=nuig_11.yaml --permissive```

We see that the conversion now runs successfully, but with a _signiciant_ number of warnings, species skipped, etc.  It is certainly worth following up on these and investigating, for any research endeavor of import!  But for today's demonstration, we will boldly forge on...